## Extending Classes

Classes offer us a good way to organize our information--both data and actions--into structures, but that alone also presents its own set of issues. For starters, what if two classes we want to make share much of their functionality (like for instance one is a specialized form of the other)? 

In that case, implementing many of these classes tends to lead to a lot of repetitive code, in turn complicating the development process and hurting readability. For instance, take the case of trying to create classes to represent `Rectangle`s and `Square`s that you'd want to draw on a screen.



In [24]:
class Rectangle {
    public int length, width;
    public double x, y;

    public Rectangle(int length, int width, double x, double y){
        System.out.println("making rectangle");
        this.length = length;
        this.width = width;
        this.x = x;
        this.y = y;
    }

    public int area() {
        return length * width;
    }

    public int perimeter() {
        return 2 * length + 2 * width;
    }
    // ... draw, change position, change size, get size, etc.
}

class Square {
    public int length;
    public double x, y;

    public Square(int length, double x, double y){
        System.out.println("making square");
        this.length = length;
        this.x = x;
        this.y = y;
    }

    public int area() {
        return length * length;
    }

    public int perimeter() {
        return 4 * length;
    }
    // ... draw, change position, change size, get size, etc.
}

Though it's a rather short example, we see that two classes are very similar in their content, both taking arguments for side lengths and positions, as well as having methods that return the area and perimeter of the object. 

In fact here, we can even see (or recall from math) that every square is just a Rectangle where the side lengths are the same (width = length).

In [25]:
Square sqr = new Square(5, 0.0, 0.0);
Rectangle sqr2 = new Rectangle(5, 5, 0.0, 0.0);

System.out.println(sqr.area());
System.out.println(sqr2.area()); // identical to above

making square
making rectangle
25
25


In these situations when we have classes A and B that share much of their implementation, but B seems to be a special case of A, we can make class B `extend` A to represent that relationship:

In [26]:
class Square extends Rectangle {
    public Square(int length, double x, double y){
        super(length, length, x, y); // must be first line in call
        System.out.println("making square"); 
    }
}

Square sqr = new Square(5,0.0,0.0);
System.out.println(sqr.area());
System.out.println(sqr.length);

making rectangle
making square
25
5


Here, by creating `Square` as a class which extends `Rectangle`, we've made our new Square class a `subclass`, or `child` class of the Rectangle class (our `superclass`), which carries with it a bunch of noticable ramifications.

#### Super

Looking at our new implementation for our new Square class, you might see that we call something called `super` instead of implementing a constructor. the 'super' here works much like the `this` construct in the previous document, although instead of calling constructors in the same class, it calls a constructor in the extended, or `parent` class. 

Here, we see that the sole constructor of `Rectangle` was used, as shown by the `making rectangle` text from the output above.

### Inheritance

You may have also noticed that when we extended Rectangle with Square, we also don't make any of the fields or methods. This is because when we created Square to extend Rectangle, it also `inherited` all of the public fields and methods of Rectangle, like `length` or `area()`, allowing our square instance to use these methods. 

Beyond using these properties and methods outside our class, they can also be used inside methods of our class too (except for private members of the superclass). 

In [31]:
class Square extends Rectangle {
    public Square(int length, double x, double y){
        super(length, length, x, y); // must be first line in call
        System.out.println("making square"); 
    }

    public int periAreaSum(){
        // using definitions from superclass
        return area() + perimeter();
    }
}

System.out.println(new Square(5,0,0).periAreaSum()); // 5*5 + 4*5 = 25 + 20 = 45

making rectangle
making square
45


Both of these consequences are quite useful for re-using common behavior, as we no longer have to provide repeat definitions for many common features, freeing up more time to work on/learn about other parts of our project .

#### Overriding Methods

Sometimes, our inherited method doesn't do exactly what we want to, either because it's too broad, or simply ill-fit for the class. For intance, we could have classes that represent an Animal and a Dog:

In [35]:
public class Animal {
    public void makeSound() {
        System.out.println("The animal makes a sound.");
    }
}

public class Dog extends Animal {
    @Override
    public void makeSound() {
        System.out.println("The dog barks.");
    }
}

Since the description for the sound animals make is a bit too broad to describe dogs' sounds, we'd want to override it to provide an implementation specifically for the Dog class. 

We do this by creating a method in our subclass `Dog` with the same `signature` (name, type, modifier) with our desired implementation. Although unnecessary, it is also good practice to add the [`@Override` annotation](https://www.baeldung.com/java-override) in order to catch mistakes for method overriding (typos, wrong type, etc).

### Subtyping 

Another consequence of inheritance is that--since a subclass inherits all the behaviors and properties of its superclass, we can use these subclasses any time we need an instance of the superclass.

That is, because a square contains all the information associated with a rectangle (a square ***is*** a rectangle), any time we need a rectangle, we can provide a square and it should work just as well. 

Therefore, we can not only assign a type of `Square` to some instance of `Square`. We should also be able to assign it the type of (or substitute its type for) `Rectangle` and it should also work fine.

In [36]:
Square s = new Square(1,0,0);
Rectangle r = new Square(2,0,0); // OK 

making rectangle
making square
making rectangle
making square


This ability for a subtype to be substituted for its supertype is eponymously known as `subtyping`, or `up-casting` (as we can think of it as an implicit type "casting" upwards in the inheritance ladder).

### (Subtype) Polymorphism

That ability to assign it the type of a superclass is also interesting at the point of usage. Since, instead of using the methods of the assigned type, it uses the methods of the underlying value's type.

Take our Dog and Animal class for example:

In [37]:
Animal a = new Animal();
Animal b = new Dog();

a.makeSound();
b.makeSound();

The animal makes a sound.
The dog barks.


Even though both objects have the given type `Animal`, their behavior differs as `a` uses the implementation of `makeSound` found in the Animal class, whereas `b` uses the implementation of the Dog class.

This ability for two identical method calls (makeSound) on two objects with identical types (Animal) to behave differently is known as `polymorphism` (literally: "many forms", as the same method takes many forms). And because this comes as a consequence of subtyping rules, it is sometimes called [`subtype polymorphism`](https://en.wikipedia.org/wiki/Subtyping) to disambiguate from the other forms of polymorphism.

## Abstract Classes

So far, we've only seen classes where all methods are explicitly implemented in their definition. However, what can we do when we want to describe something more general that doesn't have a singular implementation.

Looking back at our `Square` and `Rectangle` classes, and we can try to generalize further to `Shape`s. However we quickly see that trying to implement the `getArea` and `getPerimeter` methods presents a problem. How are we supposed to calculate the area or perimeter of some shape in general, whether it be a circle, triangle, square, rhombus, or etc? And the answer here seems to be that we can't.

So instead of racking our heads trying to think of a general implementation, we can leave those methods as unimplemented `abstract` methods, so they can be implemented when we create subclasses.

In [39]:
abstract class Shape {
    private int x, y; // position

    public void move(int dx, int dy) {
        x += dx;
        y += dy;
    }

    // Subclasses must implement these methods
    public abstract double getArea();
    public abstract double getPerimeter();
}

Notice that instead of a method body enclosed in `{}`, our abstract abstract method definitions just end with a semicolon. Likewise, notice the `abstract` modifier before `class` on the first line; this is necessary in designating our class `Shape` as one that can have some `abstract` methods, and trying to create abstract methods in a class without this modifier will produce an error. 

In [42]:
class C {
    abstract int foo();
}

CompilationException: 

## Interfaces

Another useful java concept that we haven't introduced yet are interfaces. An interface is similar to an (abstract) class, but has a few differences, namely in how inheritance, fields, and methods work.

### Interface Methods

An interface's methods are implicitly:

- `public`, so it can be accessed from anywhere
- `abstract` so they can't have an implementation

Although it is allowed (but discouraged) to explicitly use those modifiers

So the following interfaces are identical in their creation:

In [27]:
interface Runnable{
    void run(); // by default public and abstract
}

interface Runnable{
    public abstract void run();
}

Interface methods can also be `private` and `static`, although in these cases, those methods are not allowed to be abstract, so you'd need to provide a method implementation. 

Trying to define a `private abstract` method within your interface will result in a compiler error. 

In [28]:
interface Works {
    private void foo(){
        System.out.println("hello");
    }
    static void bar(){
        System.out.println("goodbye");
    }
}

In [29]:
interface Errors {
    private void foo();
}

CompilationException: 

In order to create a `public` interface method that isn't abstract, it must have the `default` modifier, showing that its supposed to be a *default* implementation for some feature described by the interface. 

In [ ]:
interface I {
    default void foo(){ // remember interface methods are public by default
        System.out.println("TODO: OVERRIDE ME!");
    }
}

#### Implementing Interfaces

instead of being `extended`, an interface must be `implemented`, like so:

In [ ]:
class C {
    public void m() {
        System.out.println("Hello, World!");
    }
}
class D extends C { }
new D().m();

Hello, World!


### Interface Motivation

interfaces at first glance seem weird. Why would we need some structure that just contains public methods that aren't implemented?

Interfaces are a useful organizational tool, they help establish a contract, or a guarantee that a class implements some behavior. In our example above, they guarantee that our cow is able to run.

Interfaces can also group classes by common behaviors, even if they don't `extend` each other. For example, for a `Runnable` we could give a cow, person, cheetah, or whatever, even though those classes aren't directly related

### Interface Fields (Constants)

Beyond methods, interfaces can also contain `fields` which are implicitly `public`, `static`, and `final`. This makes Interfaces quite useful for defining constants that we want to use across our project.

In [ ]:
interface Constants {
    // for constants, it's common practice to capitalize all letters and seperate works with an underscore
    int MINIMUM_AGE = 18;
    // some mathematical constants perhaps (Already defined in java.lang.Math)
    double PI = 3.1415926;
    double E  = 2.7182818;
    // some conversion factors maybe
    int FEET_PER_MILE = 5280;
}

System.out.println(Constants.MINIMUM_AGE);

18


In contrast, if we were to use a class, we would have to explicitly include the `public`, `static`, and `final` modifiers, leading to a lot more clutter in our code.

In [ ]:
class Constants {
    // for constants, it's common practice to capitalize all letters and seperate works with an underscore
    public static final int MINIMUM_AGE = 18;
    // some mathematical constants perhaps (Already defined in java.lang.Math)
    public static final double PI = 3.1415926;
    public static final double E  = 2.7182818;
    // some conversion factors maybe
    public static final int FEET_PER_MILE = 5280;
}

System.out.println(Constants.MINIMUM_AGE);

18


## JavaDoc Comments

JavaDoc comments, which are block comments surrounded with `/** */` (instead of the usual `/* */`) a special kind of comment that gives documentation about that class/method. Though as a comment it makes no difference in code, it can be usedful for describing what a method or class does in detail.

The extensions that we use in VSCode also allow us to hover over a method or class and see a pop-up with it's javadoc description.

In [ ]:
/**
 * This is a class which does certain things
 * @author me myself I
 */
class C {
    /**
     * adds two provided inputs and returns the result
     * @param a first summand
     * @param b second summand
     * @return the sum of the two inputs
     */
    public static int add(int a, int b) {
        return a + b;
    }
}

## Generics And Type Parameters

## Enumerations (Enums)

## Function Expressions (Lambdas)